In [48]:
import optimal_tap_position as otp
import pandas as pd
from power_grid_model.utils import json_deserialize_from_file
import numpy as np

In [49]:
input_data = json_deserialize_from_file("input_network_data1.json")

print(type(input_data))
print(pd.DataFrame(input_data.keys()))

<class 'dict'>
             0
0         node
1         line
2  transformer
3       source
4     sym_load


In [50]:
print(pd.DataFrame(input_data['node']))

   id  u_rated
0   0  10500.0
1   1    400.0
2   2    400.0
3   3    400.0
4   4    400.0
5   5    400.0
6   6    400.0
7   7    400.0
8   8    400.0
9   9    400.0


In [51]:
print(pd.DataFrame(input_data['line']))

   id  from_node  to_node  from_status  to_status        r1        x1  \
0  16          1        2            1          1  0.000310  0.000141   
1  17          2        3            1          1  0.000052  0.000007   
2  18          2        4            1          1  0.000330  0.000150   
3  19          4        5            1          1  0.000056  0.000008   
4  20          1        6            1          1  0.000316  0.000144   
5  21          6        7            1          1  0.000052  0.000007   
6  22          6        8            1          1  0.000309  0.000140   
7  23          8        9            1          1  0.000062  0.000009   
8  24          4        8            1          0  0.000360  0.000164   

             c1   tan1        r0        x0            c0   tan0           i_n  
0  1.406945e-05  0.003  0.001121  0.000387  8.360108e-06  0.001  4.485912e+06  
1  5.235094e-07  0.003  0.000175  0.000020  3.102278e-07  0.001  9.694619e+04  
2  1.498785e-05  0.003  0.001

In [52]:
print(pd.DataFrame(input_data['transformer']))

   id  from_node  to_node  from_status  to_status       u1     u2        sn  \
0  11          0        1            1          1  10750.0  420.0  630000.0   

      uk      pk  ...  tap_nom  tap_size  uk_min  uk_max  pk_min  pk_max  \
0  0.041  5200.0  ...        3     250.0     NaN     NaN     NaN     NaN   

   r_grounding_from  x_grounding_from  r_grounding_to  x_grounding_to  
0               NaN               NaN             0.0             0.0  

[1 rows x 29 columns]


In [53]:
print(pd.DataFrame(input_data['source']))

   id  node  status  u_ref  u_ref_angle           sk  rx_ratio  z01_ratio
0  10     0       1   1.05          NaN  200000000.0       NaN        NaN


In [54]:
print(pd.DataFrame(input_data['sym_load']))

   id  node  status  type  p_specified  q_specified
0  12     3       1     0          0.0          0.0
1  13     5       1     0          0.0          0.0
2  14     7       1     0          0.0          0.0
3  15     9       1     0          0.0          0.0


In [55]:
active_power_data = pd.read_parquet('active_power_profile1.parquet')
print(type(active_power_data))

print(active_power_data.index.tolist())

<class 'pandas.core.frame.DataFrame'>
[Timestamp('2025-01-01 00:00:00'), Timestamp('2025-01-01 00:15:00'), Timestamp('2025-01-01 00:30:00'), Timestamp('2025-01-01 00:45:00'), Timestamp('2025-01-01 01:00:00'), Timestamp('2025-01-01 01:15:00'), Timestamp('2025-01-01 01:30:00'), Timestamp('2025-01-01 01:45:00'), Timestamp('2025-01-01 02:00:00'), Timestamp('2025-01-01 02:15:00'), Timestamp('2025-01-01 02:30:00'), Timestamp('2025-01-01 02:45:00'), Timestamp('2025-01-01 03:00:00'), Timestamp('2025-01-01 03:15:00'), Timestamp('2025-01-01 03:30:00'), Timestamp('2025-01-01 03:45:00'), Timestamp('2025-01-01 04:00:00'), Timestamp('2025-01-01 04:15:00'), Timestamp('2025-01-01 04:30:00'), Timestamp('2025-01-01 04:45:00'), Timestamp('2025-01-01 05:00:00'), Timestamp('2025-01-01 05:15:00'), Timestamp('2025-01-01 05:30:00'), Timestamp('2025-01-01 05:45:00'), Timestamp('2025-01-01 06:00:00'), Timestamp('2025-01-01 06:15:00'), Timestamp('2025-01-01 06:30:00'), Timestamp('2025-01-01 06:45:00'), Timestamp

In [56]:
reactive_power_data = pd.read_parquet('active_power_profile1.parquet')
print(type(reactive_power_data))

print(reactive_power_data)

<class 'pandas.core.frame.DataFrame'>
Load ID                        12            13            14            15
Timestamp                                                                  
2025-01-01 00:00:00  53845.936311  52868.809400  59437.379259  61193.999654
2025-01-01 00:15:00  58623.012703  66992.056154  57797.477123  48956.385676
2025-01-01 00:30:00  64800.016277  54713.600702  51293.354771  58092.153016
2025-01-01 00:45:00  61525.469129  59903.401304  49747.760990  62055.246675
2025-01-01 01:00:00  60289.133403  56883.894966  56301.650801  56226.510409
...                           ...           ...           ...           ...
2025-01-10 22:45:00  69484.715910  71348.849463  47979.322152  64352.546069
2025-01-10 23:00:00  61503.570916  60836.586259  76135.394978  59164.472679
2025-01-10 23:15:00  56045.155706  54275.886219  63557.959847  66834.711500
2025-01-10 23:30:00  73370.995338  59556.779716  57056.774548  62640.438159
2025-01-10 23:45:00  62018.984783  58840.504085  7

In [145]:
grid = otp.OptimalTapPosition(grid_data=input_data)

output_data = grid.batch_powerflow(active_power_profile1=active_power_data, reactive_power_profile1=reactive_power_data)

a = {}
tap_pro = {}
for i in range(len(output_data["node"]["id"])):
    if isinstance(output_data["node"]["id"][i], (list, np.ndarray)):
        for j in range(len(output_data["node"]["id"][i])):
            if output_data["node"]["id"][i][j] == 0:
                a[i] = output_data["node"]["u"][i][j]
    elif output_data["node"]["id"][i] == 0:
        a[i] = output_data["node"]["u"][i]

for i in range(len(a)):
    tap_pro[i] = ((a[i] - input_data["transformer"]["u1"])/input_data["transformer"]["u1"])*100
    print(tap_pro[i])

print(len(tap_pro))


[2.43807789]
[2.43537079]
[2.43724164]
[2.43490506]
[2.43680949]
[2.43327299]
[2.42762441]
[2.41827018]
[2.41455483]
[2.41157172]
[2.40310247]
[2.40875776]
[2.3894382]
[2.37846772]
[2.40349574]
[2.38589159]
[2.39230817]
[2.36366231]
[2.35127879]
[2.34754073]
[2.33834681]
[2.35222497]
[2.36183409]
[2.35289144]
[2.37255998]
[2.35676025]
[2.37135357]
[2.33578964]
[2.35226255]
[2.35515114]
[2.36310741]
[2.34861108]
[2.35767719]
[2.35198289]
[2.37559674]
[2.35062795]
[2.34909955]
[2.34896237]
[2.33512767]
[2.35797257]
[2.36642412]
[2.36438548]
[2.3551062]
[2.37627765]
[2.35600066]
[2.3433057]
[2.35694541]
[2.36049089]
[2.34826107]
[2.34678241]
[2.34087466]
[2.35528833]
[2.3297115]
[2.36709919]
[2.37291856]
[2.37470316]
[2.36085371]
[2.38005921]
[2.3745347]
[2.38032575]
[2.37953259]
[2.38171287]
[2.36587914]
[2.38651592]
[2.38432498]
[2.37945983]
[2.36569273]
[2.38673925]
[2.36505695]
[2.4007631]
[2.37227939]
[2.39015963]
[2.38076779]
[2.3949754]
[2.39186001]
[2.41717091]
[2.407854]
[2.41259

In [144]:
print(len(tap_pro))

tap_table = pd.DataFrame()
 
tap_table["Timestamp"] = active_power_data.index.tolist()
tap_table["Tap"] = tap_pro


print(tap_table)


1


ValueError: Length of values (1) does not match length of index (960)

In [58]:
print(grid.aggregate_voltage_table(active_power_profile1=active_power_data, reactive_power_profile1=reactive_power_data))

              Timestamp  max_id  u_pu_max  min_id  u_pu_min
0   2025-01-01 00:00:00       1  1.057378       0  1.048771
1   2025-01-01 00:15:00       1  1.056936       0  1.048743
2   2025-01-01 00:30:00       1  1.057242       0  1.048762
3   2025-01-01 00:45:00       1  1.056860       0  1.048738
4   2025-01-01 01:00:00       1  1.057171       0  1.048758
..                  ...     ...       ...     ...       ...
955 2025-01-10 22:45:00       1  1.055101       0  1.048628
956 2025-01-10 23:00:00       1  1.054705       0  1.048603
957 2025-01-10 23:15:00       1  1.056201       0  1.048697
958 2025-01-10 23:30:00       1  1.055149       0  1.048631
959 2025-01-10 23:45:00       1  1.054968       0  1.048620

[960 rows x 5 columns]


In [68]:
voltage_table = pd.DataFrame()

voltage_table["Timestamp"] = active_power_data.index.tolist()
voltage_table["max_id"] = output_data["node"][:, pd.DataFrame(output_data["node"]["u_pu"][:, :]).idxmax(axis=1).tolist() ]["id"][0, :]
voltage_table["u_pu_max"] = pd.DataFrame(output_data["node"]["u_pu"][:, :]).max(axis=1).tolist()
voltage_table["min_id"] = output_data["node"][:, pd.DataFrame(output_data["node"]["u_pu"][:, :]).idxmin(axis=1).tolist() ]["id"][0, :]
voltage_table["u_pu_min"] = pd.DataFrame(output_data["node"]["u_pu"][:, :]).min(axis=1).tolist()

print(pd.DataFrame(voltage_table))

              Timestamp  max_id  u_pu_max  min_id  u_pu_min
0   2025-01-01 00:00:00       1  1.057378       0  1.048771
1   2025-01-01 00:15:00       1  1.056936       0  1.048743
2   2025-01-01 00:30:00       1  1.057242       0  1.048762
3   2025-01-01 00:45:00       1  1.056860       0  1.048738
4   2025-01-01 01:00:00       1  1.057171       0  1.048758
..                  ...     ...       ...     ...       ...
955 2025-01-10 22:45:00       1  1.055101       0  1.048628
956 2025-01-10 23:00:00       1  1.054705       0  1.048603
957 2025-01-10 23:15:00       1  1.056201       0  1.048697
958 2025-01-10 23:30:00       1  1.055149       0  1.048631
959 2025-01-10 23:45:00       1  1.054968       0  1.048620

[960 rows x 5 columns]


In [71]:
print(pd.DataFrame(voltage_table["u_pu_max"]))

     u_pu_max
0    1.057378
1    1.056936
2    1.057242
3    1.056860
4    1.057171
..        ...
955  1.055101
956  1.054705
957  1.056201
958  1.055149
959  1.054968

[960 rows x 1 columns]


In [95]:
tap = 0
for i in range(len(voltage_table)):
    u_pu_max = voltage_table["u_pu_max"][i]
    u_pu_min = voltage_table["u_pu_min"][i]
    tap_procent_max = (u_pu_max-1)/1*100
    tap_procent_min = (u_pu_min-1)/1*100
    tap = tap_procent_max + tap_procent_min + tap

tap_average_deviation = tap/(2*len(voltage_table))
print(tap_average_deviation)

5.305535012364462


In [88]:
for i in range(len(voltage_table)):
    a = voltage_table["u_pu_max"][i]
    tap_procent_min = (a-1)/1*100
    print(tap_procent_min)

5.7378243077615965
5.693619700512853
5.724168377940031
5.686015423069457
5.717111603287073
5.659370799031294
5.567185224021065
5.41461724069352
5.354052218709482
5.305437455867201
5.167483650056526
5.2595922688113195
4.945119448011992
4.816050280246764
5.173888820484418
4.887441862886255
4.991802995758876
4.8008923684540905
4.7882140017562635
4.784386938134477
4.774974111897645
4.789182710065254
4.799020616871652
4.789865042951691
4.810001879392001
4.79382597004252
4.808766746730764
4.772356059599137
4.789221179717518
4.792178551762372
4.800324254731403
4.785482772381644
4.794764739965274
4.788934858757976
4.813110949402244
4.787547659023295
4.785982871815886
4.785842426221487
4.771678329376905
4.795067158019717
4.80371992945583
4.801632752140517
4.792132541218175
4.813808070355985
4.793048293429658
4.780051068849578
4.79401553395542
4.797645430429109
4.785124431261489
4.783610565173468
4.777562151707926
4.79231900035022
4.766133206457623
4.80441107402958
4.81036899907743
4.81219608937